In [23]:
string='hello' ###짧은 문자열
long_string='oh! hello! my name is ___' ###긴문자열
splitter='!' ###구분자
def String_length(string): ###길이를 계산해주는 함수 
    length=0
    for i in string:
        length+=1
    return length
def Find_string_index(string,long_string): #### 긴문자열에서 짧은 문자열  위치 찾아주는 함수
    for i in range(String_length(long_string)):
        if long_string[i:i+String_length(string)]==string:
            return i
def split_string(string,splitter): ###긴문자열을 구분자로 나눠주는 함수
    array=[]
    save_point=0
    for i in range(String_length(string)):
        if string[i]==splitter:
            array.append(string[save_point:i])
            save_point=i
    array.append(string[save_point:i])
    return array

string='hello' ###짧은 문자열
long_string='oh! hello! my name is ___' ###긴문자열
splitter='!' ###구분자
print(String_length(string))

print(Find_string_index(string,long_string))

print(split_string(long_string,splitter))
    

5
4
['oh', '! hello', '! my name is __']


In [ ]:
tf2.

In [34]:
# Lab 11 MNIST and Convolutional Neural Network
import tensorflow as tf  ###텐서플로를 불러옴
import random  ###랜덤넘버 생성용
# import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data  ### mnist 데이터 가져옴

tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True) ###mnist 데이터셋
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset

# hyper parameters
learning_rate = 0.001  ###학습률 지정
training_epochs = 2  ###15번의 에포크
batch_size = 15  ###배치사이즈는 100

# input place holders
X = tf.placeholder(tf.float32, [None, 784])   ###하나의 이미지가 784차원이니까 이렇게 쓴다
X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)
Y = tf.placeholder(tf.float32, [None, 10]) ### 레이블 값 입력

# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 16], stddev=0.01))  ####첫레이어에선 채널크기를 32개로 늘려준다
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME') ###convolutinon layer
L1 = tf.nn.relu(L1) ##활성화 함수는 relu 함수
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME') ### 2*2 maxpooling
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
'''

# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

W100=tf.Variable(tf.random_normal([1, 1, 32, 32], stddev=0.01))
L2 = tf.nn.conv2d(L2, W100, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)

L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')

###세번째 레이어 만들기

W3=tf.Variable(tf.random_normal([3,3,32,64]))
L3=tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding='SAME')
L3=tf.nn.relu(L3)
L3_flat = tf.reshape(L3, [-1, 7 * 7 * 64])
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
'''

# Final FC 7x7x64 inputs -> 10 outputs
W4 = tf.Variable(tf.random_normal([7 * 7 * 64, 10]))
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L3_flat, W4) + b

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist.test.images, Y: mnist.test.labels}))

# Get one and predict
r = random.randint(0, mnist.test.num_examples - 1)
print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Prediction: ", sess.run(
    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1]}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started. It takes sometime.
Epoch: 0001 cost = 0.234187042
Epoch: 0002 cost = 0.085748067
Learning Finished!
Accuracy: 0.9786
Label:  [9]
Prediction:  [9]
